# 創建一個sqlite檔案，整理抓到的csv檔案

* 利用 glob及 pandas套件來讀取資料

* 建立一個 sqlite檔案 來存入資料庫

* 進一步整理為用個股來分的資料庫

## 1. 讀取我們抓到的資料
### 用 pandas一個一個讀入很沒有效率，使用 glob套件一次讀入所有的 csv檔案

In [2]:
import pandas as pd
import sqlite3
import glob

In [3]:
# glob 套件是用來找符合特定規則的文件名，跟我們用搜尋跳出來的結果差不多，這邊我們查詢附檔名為csv的檔案並存為一個列表的形式
all_csv_file = glob.glob('*.csv') 

In [4]:
all_csv_file

['20201117.csv', '20201118.csv', '20201119.csv', '20201120.csv']

In [6]:
df = pd.read_csv(all_csv_file[0])
df.iloc[:, 1:].head()

,證券代號,證券名稱,成交股數,成交筆數,成交金額,開盤價,最高價,最低價,收盤價,漲跌(+/-),漲跌價差,最後揭示買價,最後揭示買量,最後揭示賣價,最後揭示賣量,本益比
0,0050,元大台灣50,"8,297,028","6,274","945,842,517",115.00,115.75,113.40,113.60,+,0.80,113.55,14,113.60,264,0.00
1,0051,元大中型100,"52,504",46,"2,155,141",40.86,41.25,40.86,41.04,+,0.20,40.97,35,41.05,2,0.00
2,0052,富邦科技,"1,625,944",798,"164,040,093",102.60,103.00,100.05,100.05,+,1.05,100.05,1,100.10,12,0.00
3,0053,元大電子,"52,574",41,"2,854,523",54.55,54.75,53.95,53.95,+,0.70,53.90,2,54.00,2,0.00
4,0054,元大台商50,"33,877",13,"901,322",26.70,26.70,26.59,26.61,+,0.09,26.61,26,26.70,10,0.00


## 2. 創建資料庫，存成以時間排序為一張張表的資料庫
* python 內建sqlite套件，我們無須特別安裝
* 支援完整sql語法查詢我們的資料
* 使用以及轉移方便，一個資料庫就像一個本地文件

In [9]:
dbname = 'twstock.db'
db = sqlite3.connect(dbname)
for file_name in all_csv_file:
    pd.read_csv(file_name).iloc[:, 1:].to_sql(file_name.replace(".csv",''), db, if_exists = 'replace')

## 如何讀取資料庫的表格
### 簡單介紹如何讀取sqlite檔案裡面的表格

In [22]:
all_csv_file[1]

'20201118.csv'

In [27]:
test_df = pd.read_sql(con= db, sql = 'SELECT * FROM "20201117"')
test_df['Date'] = '20201117'
test_df_2 = pd.read_sql(con = db, sql = 'SELECT * FROM "20201118"')
test_df_2['Date'] = '20201118'

In [28]:
test_dict = dict(tuple(test_df.append(test_df_2).groupby('證券代號')))

In [30]:
test_2330 = test_dict['2330']
test_2330['Date'] = pd.to_datetime(test_2330['Date'])

In [31]:
test_2330

,index,證券代號,證券名稱,成交股數,成交筆數,成交金額,開盤價,最高價,最低價,收盤價,漲跌(+/-),漲跌價差,最後揭示買價,最後揭示買量,最後揭示賣價,最後揭示賣量,本益比,Date
422,422,2330,台積電,"75,763,870","68,560","37,446,278,783",502.00,506.00,485.50,485.50,+,1.5,485.50,377,486.00,197,25.63,2020-11-17
422,422,2330,台積電,"56,607,501","38,186","27,883,387,314",490.00,497.00,486.50,497.00,+,11.5,496.50,276,497.00,131,26.24,2020-11-18


## 3. 整理出以個股為一張張表的資料庫

In [16]:
dates_list = [file_name.replace('.csv', '') for file_name in all_csv_file]

In [20]:
%%time
## 先做成一個大表
total_df = pd.DataFrame()
for date in dates_list:
    df = pd.read_sql(con = db, sql = 'SELECT * FROM '+ '"'+ date + '"')
    df["Date"] = date
    total_df = total_df.append(df)

Wall time: 77.6 ms


In [21]:
total_df.shape

(4480, 18)

In [32]:
dbname_2 = "twstock2"
db2 = sqlite3.connect(dbname_2)

In [33]:
total_dict = dict(tuple(total_df.groupby('證券代號')))
for key in total_dict.keys():
    df = total_dict[key].iloc[:, 2:]
    df['Date'] =pd.to_datetime(df['Date'])
    df = df.sort_values(by = ['Date'])
    df.to_sql(key, db2, if_exists = 'replace')

In [42]:
pd.read_sql(con = db2, sql = 'SELECT * FROM "2939"')

,index,證券名稱,成交股數,成交筆數,成交金額,開盤價,最高價,最低價,收盤價,漲跌(+/-),漲跌價差,最後揭示買價,最後揭示買量,最後揭示賣價,最後揭示賣量,本益比,Date
0,665,凱羿-KY,"2,162,406",796,"183,957,563",81.20,89.30,81.20,84.60,+,3.4,84.60,1,85.80,1,0.00,2020-11-17 00:00:00
1,665,凱羿-KY,"277,945",153,"21,179,409",76.20,76.20,76.20,76.20,-,8.4,--,0,76.20,512,0.00,2020-11-18 00:00:00
2,664,凱羿-KY,"3,637,386","1,862","264,481,652",76.00,78.60,69.00,71.00,-,5.2,71.00,14,71.10,2,0.00,2020-11-19 00:00:00
3,664,凱羿-KY,"2,443,392","1,522","161,398,105",71.00,73.00,63.90,63.90,-,7.1,63.90,38,64.00,14,0.00,2020-11-20 00:00:00
